In [20]:
# this notebook is for developing the speech_recognition system
from __future__ import unicode_literals
import math
import random
import spacy
import time
import csv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import wave, sys, pyaudio
import speech_recognition as sr
import pyttsx3
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [21]:
#this code was taken from the voiceshell.py program

"""
def roboVoice(statement):
    engine = pyttsx3.init();
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate-25)
    voices = engine.getProperty('voices')
    engine.setProperty('voice', 'en-uk-rp')
    engine.say(statement)
    engine.runAndWait()
"""

print("Voiceshell starting...")
# roboVoice("Ahem. Starting...")
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

def sentvec(s):
    sent = nlp(s)
    word_vectors = [w.vector for w in sent]
    return meanv(word_vectors)

def spacy_closest_sent(space, input_str, n=1):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec), reverse=True)[:n]

nlp = spacy.load('en')


Voiceshell starting...


In [16]:
# more code taken from voiceshell.py
print("Loading text...")
# roboVoice("Loading text files...")
corpusFile = 'voiceshell_audio_LUT.csv'

with open(corpusFile, 'r') as f:
    reader = csv.reader(f)
    all_lines = list(reader)

audio_lookup_table = {}
sentences = []
for line in all_lines:
    sentence = line[0]
    if not sentence.isspace():
        sentences.append(nlp(sentence))
    filename = line[1]
    audio_lookup_table[sentence] = filename
print("Done.")
# roboVoice("Starting speech recognition...")
print("Starting up speech recognition...")
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Calibrating microphone for ambient noise... (this will take 5 seconds)")
    # roboVoice("Calibrating microphone for ambient noise... (this will take 5 seconds)")
    r.adjust_for_ambient_noise(source, duration=5)
    print("Done, starting program.")
    # roboVoice("Done, starting program.")

Loading text...
Done.
Starting up speech recognition...
Calibrating microphone for ambient noise... (this will take 5 seconds)
Done, starting program.


In [17]:
# a higher value equates to less sensitivity, a lower value is more sensitive: 50-4000 are typical values
# this is the manual way to adjust the sensitivity while adjust_for_ambiant_noise is the automatic way
r.energy_threshold = 4000

In [29]:
# taken from voiceshell.py but now tries google if google is down then it uses sphinx
def runLoop():
    # roboVoice("I'm listening...")
    print("Say something!")

    with sr.Microphone() as source:
        audio = r.listen(source)
    
    # try with google first
    try:
        userInput = r.recognize_google(audio)
        print("Google thinks you said : ", userInput)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")

    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        try:
            userInput = r.recognize_sphinx(audio)
            print("Sphinx thinks you said")
        except sr.UnknownValueError:
            # roboVoice("Sorry, I couldn't understand that.")
            print("Sorry, I didn't understand that.")
        except sr.RequestError as e:
            # roboVoice("Sphinx Error {0}".format(e))
            print("Sphinx recognition error; {0}".format(e))
    
    matched = process.extractBests(userInput, sentences, score_cutoff = 60)

    if(userInput == 'open the pod bay doors'):
        audioFile = 'Audio/sorry.wav'

    elif not matched:
        for sent in spacy_closest_sent(sentences, userInput):
            output = sent.text
            print(output)

        audioFile = audio_lookup_table[output]
        print(audioFile)

    else:
        output = random.choice(matched)
        cleaned_output = str(output[0])
        print(cleaned_output)

        audioFile = audio_lookup_table[cleaned_output]
        print(audioFile)

    sound = wave.open(audioFile)
    p = pyaudio.PyAudio()
    chunk = 1024
    stream = p.open(format = p.get_format_from_width(sound.getsampwidth()), channels = sound.getnchannels(), rate = sound.getframerate(), output = True)
    data = sound.readframes(chunk)
    while len(data) > 0:
        stream.write(data)
        data = sound.readframes(chunk)
    stream.stop_stream()
    stream.close()

    p.terminate



In [30]:
for i in range(0,9):
    runLoop()

Say something!
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 8] nodename nor servname provided, or not known
you said :  that will have
But his eyes are so glad-pale-lashed, green eyes- that I forget to question his awareness.
Audio/AmyBrown/AmyBrown42.wav
Say something!
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 8] nodename nor servname provided, or not known
you said :  if that
like scales that tilt toward injustice.
Audio/BobOrr/BobOrr7.wav
Say something!
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 8] nodename nor servname provided, or not known
you said :  okay i can sing something now
At times I hate my wide painted eyes, though I'm becoming wiser.
Audio/AmyBrown/AmyBrown26.wav
Say something!
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 8] nodenam

IndexError: list index out of range

In [22]:
import sys, os
from pocketsphinx.pocketsphinx import *
from sphinxbase.sphinxbase import *
import pyaudio

modeldir = "../../../model"
datadir = "../../../test/data"

# Create a decoder with certain model
config = Decoder.default_config()
config.set_string('-hmm', os.path.join(modeldir, 'en-us/en-us'))
config.set_string('-dict', os.path.join(modeldir, 'en-us/cmudict-en-us.dict'))
config.set_string('-keyphrase', 'forward')
config.set_float('-kws_threshold', 1e+20)


p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=1024)
stream.start_stream()

# Process audio chunk by chunk. On keyword detected perform action and restart search
decoder = Decoder(config)
decoder.start_utt()
while True:
    buf = stream.read(1024)
    if buf:
         decoder.process_raw(buf, False, False)
    else:
         break
    if decoder.hyp() != None:
        print ([(seg.word, seg.prob, seg.start_frame, seg.end_frame) for seg in decoder.seg()])
        print ("Detected keyword, restarting search")
        decoder.end_utt()
        decoder.start_utt()

RuntimeError: new_Decoder returned -1

In [26]:
# google's system works much better, I think we should try that 
# obtain audio from the microphone
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source)
    
try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
    print("Google Speech Recognition thinks you said " + r.recognize_google(audio))
    print("Sphinx thinks you said " + r.recognize_sphinx(audio))
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

Say something!
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 8] nodename nor servname provided, or not known
